In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(0)

In [3]:
from Server import  Server
from Clients import Client
from Train import Trainer

In [4]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=12)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)

parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--ensemble_flag', type=bool, default=True)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:2' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model12/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
args_train = parser_train.parse_args(args=[])


In [5]:
seed_everything(0)

In [6]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.33it/s]


In [7]:
server = Server(args_train,clients)

In [8]:
'''import pickle
scale=12
# Load the server object
with open('../result/'+str(scale)+'/server_benchmark.pkl', 'rb') as f:
    server_benchmark = pickle.load(f)

# Load the clients object
with open('../result/'+str(scale)+'/clients_benchmark.pkl', 'rb') as f:
    clients_benchmark = pickle.load(f)


for i in range(7):
    clients[i].copy_client(clients_benchmark[i])

server = Server(args_train,clients)'''

"import pickle\nscale=12\n# Load the server object\nwith open('../result/'+str(scale)+'/server_benchmark.pkl', 'rb') as f:\n    server_benchmark = pickle.load(f)\n\n# Load the clients object\nwith open('../result/'+str(scale)+'/clients_benchmark.pkl', 'rb') as f:\n    clients_benchmark = pickle.load(f)\n\n\nfor i in range(7):\n    clients[i].copy_client(clients_benchmark[i])\n\nserver = Server(args_train,clients)"

In [9]:
server.fed_train()

test performance: [0.1677448276226243, 0.16753428755965952, 0.19564487713657014, 0.17997160227331396, 0.17244937605135244, 0.18444072286764238, 0.18148799113010708]
Epoch: 0 | Loss: 0.0867
Epoch: 0 | Loss: 0.0716
Epoch: 0 | Loss: 0.1119
Epoch: 0 | Loss: 0.0911
Epoch: 0 | Loss: 0.0706
Epoch: 0 | Loss: 0.0866
Epoch: 0 | Loss: 0.0930
Federated training Epoch [1/200] Val Loss: 0.0754
test performance: [0.07842033547795799, 0.08361714192959543, 0.09718022554194274, 0.08820951684084657, 0.09092676126691576, 0.08746098989799414, 0.09202577464588702]
Epoch: 0 | Loss: 0.0747
Epoch: 0 | Loss: 0.0751
Epoch: 0 | Loss: 0.0923
Epoch: 0 | Loss: 0.0844
Epoch: 0 | Loss: 0.0757
Epoch: 0 | Loss: 0.0781
Epoch: 0 | Loss: 0.0920
Federated training Epoch [2/200] Val Loss: 0.0736
test performance: [0.07694024429337619, 0.08226170071898257, 0.0935047646244503, 0.0861237039917136, 0.08941671346658714, 0.08499742643780088, 0.08936039526780991]
Epoch: 0 | Loss: 0.0620
Epoch: 0 | Loss: 0.0757
Epoch: 0 | Loss: 0.07

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [10]:
server.central_train()

Epoch [1/200] Train Loss: 0.0941 Val Loss: 0.0752
Epoch [2/200] Train Loss: 0.0763 Val Loss: 0.0731
Epoch [3/200] Train Loss: 0.0730 Val Loss: 0.0701
Epoch [4/200] Train Loss: 0.0701 Val Loss: 0.0685
Epoch [5/200] Train Loss: 0.0689 Val Loss: 0.0680
Epoch [6/200] Train Loss: 0.0682 Val Loss: 0.0678
Epoch [7/200] Train Loss: 0.0676 Val Loss: 0.0669
Epoch [8/200] Train Loss: 0.0670 Val Loss: 0.0661
Epoch [9/200] Train Loss: 0.0665 Val Loss: 0.0657
Epoch [10/200] Train Loss: 0.0663 Val Loss: 0.0653
Epoch [11/200] Train Loss: 0.0658 Val Loss: 0.0650
Epoch [12/200] Train Loss: 0.0654 Val Loss: 0.0649
Epoch [13/200] Train Loss: 0.0650 Val Loss: 0.0646
Epoch [14/200] Train Loss: 0.0645 Val Loss: 0.0641
Epoch [15/200] Train Loss: 0.0643 Val Loss: 0.0635
Epoch [16/200] Train Loss: 0.0638 Val Loss: 0.0633
Epoch [17/200] Train Loss: 0.0635 Val Loss: 0.0629
Epoch [18/200] Train Loss: 0.0632 Val Loss: 0.0631
Epoch [19/200] Train Loss: 0.0628 Val Loss: 0.0628
Epoch [20/200] Train Loss: 0.0625 Val Lo

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [11]:
server.local_train()

Launch Local Training!
Epoch [1/200] Train Loss: 0.0877 Val Loss: 0.0745
Epoch [2/200] Train Loss: 0.0747 Val Loss: 0.0717
Epoch [3/200] Train Loss: 0.0713 Val Loss: 0.0692
Epoch [4/200] Train Loss: 0.0695 Val Loss: 0.0683
Epoch [5/200] Train Loss: 0.0685 Val Loss: 0.0680
Epoch [6/200] Train Loss: 0.0680 Val Loss: 0.0675
Epoch [7/200] Train Loss: 0.0672 Val Loss: 0.0667
Epoch [8/200] Train Loss: 0.0667 Val Loss: 0.0658
Epoch [9/200] Train Loss: 0.0661 Val Loss: 0.0653
Epoch [10/200] Train Loss: 0.0656 Val Loss: 0.0657
Epoch [11/200] Train Loss: 0.0651 Val Loss: 0.0654
Epoch [12/200] Train Loss: 0.0646 Val Loss: 0.0638
Epoch [13/200] Train Loss: 0.0642 Val Loss: 0.0640
Epoch [14/200] Train Loss: 0.0636 Val Loss: 0.0630
Epoch [15/200] Train Loss: 0.0633 Val Loss: 0.0627
Epoch [16/200] Train Loss: 0.0627 Val Loss: 0.0628
Epoch [17/200] Train Loss: 0.0622 Val Loss: 0.0619
Epoch [18/200] Train Loss: 0.0618 Val Loss: 0.0621
Epoch [19/200] Train Loss: 0.0615 Val Loss: 0.0611
Epoch [20/200] Tr

In [12]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune()
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0579 Val Loss: 0.0578
Epoch [2/20] Train Loss: 0.0576 Val Loss: 0.0572
Epoch [3/20] Train Loss: 0.0573 Val Loss: 0.0570
Epoch [4/20] Train Loss: 0.0571 Val Loss: 0.0570
Epoch [5/20] Train Loss: 0.0569 Val Loss: 0.0569
Epoch [6/20] Train Loss: 0.0568 Val Loss: 0.0568
Epoch [7/20] Train Loss: 0.0567 Val Loss: 0.0567
Epoch [8/20] Train Loss: 0.0566 Val Loss: 0.0566
Epoch [9/20] Train Loss: 0.0565 Val Loss: 0.0566
Epoch [10/20] Train Loss: 0.0564 Val Loss: 0.0566
Epoch [11/20] Train Loss: 0.0563 Val Loss: 0.0564
Epoch [12/20] Train Loss: 0.0563 Val Loss: 0.0564
Epoch [13/20] Train Loss: 0.0562 Val Loss: 0.0565
Epoch [14/20] Train Loss: 0.0561 Val Loss: 0.0562
Epoch [15/20] Train Loss: 0.0560 Val Loss: 0.0563
Epoch [16/20] Train Loss: 0.0559 Val Loss: 0.0561
Epoch [17/20] Train Loss: 0.0559 Val Loss: 0.0560
Epoch [18/20] Train Loss: 0.0558 Val Loss: 0.0562
Epoch [19/20] Train Loss: 0.0558 Val Loss: 0.0559
Epoch [20/20] Train Loss: 0.0557 Val Loss: 0.0559
1

In [13]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss)
    fed_local_preds.append(fed_local_pred)
    fed_local_models.append(fed_local_model)
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.06865391036980364, 0.08005914130337434, 0.08350040195892526, 0.0766893360215201, 0.07898468616074078, 0.07257786304498576, 0.07864485521583933]
0.07701574201074132


In [14]:
local_losses=[]
local_preds=[]
local_models=[]
for i in range(args_train.number_clients):
    local_pred,local_loss,local_model=clients[i].local_evaluation()
    local_losses.append(local_loss)
    local_preds.append(local_pred)
    local_models.append(local_model)
print(local_losses)

[0.08187799558859982, 0.08105895224295251, 0.08582551417591637, 0.07698124492770597, 0.07616442277364127, 0.07639797528159536, 0.09301163217894835]


In [15]:
central_losses=[]
central_preds=[]
central_models=[]
for i in range(args_train.number_clients):
    central_pred,central_loss,central_model=server.central_evaluation(dataset=i)
    central_losses.append(central_loss)
    central_preds.append(central_pred)
    central_models.append(central_model)
print(central_losses)

[0.06877125395232275, 0.08701450000070546, 0.08665346308317903, 0.07787583673959725, 0.08269334102228079, 0.0758516240788445, 0.08388754275104363]


In [16]:
print(local_losses)
print(central_losses)
print(fed_local_losses)
print(local_fine_tune_losses)

[0.08187799558859982, 0.08105895224295251, 0.08582551417591637, 0.07698124492770597, 0.07616442277364127, 0.07639797528159536, 0.09301163217894835]
[0.06877125395232275, 0.08701450000070546, 0.08665346308317903, 0.07787583673959725, 0.08269334102228079, 0.0758516240788445, 0.08388754275104363]
[0.06865391036980364, 0.08005914130337434, 0.08350040195892526, 0.0766893360215201, 0.07898468616074078, 0.07257786304498576, 0.07864485521583933]
[0.0674807617747008, 0.07917149850384217, 0.08372500471209418, 0.07453125617658235, 0.07562352281880297, 0.07363936380317358, 0.0791351490347863]


In [38]:
df = pd.DataFrame({
    'local_losses': local_losses,
    'central_losses': central_losses,
    'fed_local_losses': fed_local_losses,
    'local_fine_tune_losses': local_fine_tune_losses
})
df.T.to_csv('losses.csv', index=False)

In [64]:
actual_y_lst=[]

for i in range(args_train.number_clients):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths='wf'+str(i+1)
    test_data, test_loader = get_data(args_temp,flag='test')
    actual_y=[]
    for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
        actual_y.append(seq_y)
    actual_y = torch.cat([torch.flatten(t) for t in actual_y])
    actual_y_lst.append(actual_y)


In [17]:
import pickle

# Save the server object
with open('../result/12/server_benchmark.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/12/clients_benchmark.pkl', 'wb') as f:
    pickle.dump(clients, f)

: 

In [334]:
import pickle

# Save server and clients
with open('server.pkl', 'wb') as f:
    pickle.dump(server, f)

with open('clients.pkl', 'wb') as f:
    pickle.dump(clients, f)

# Load server and clients
with open('server.pkl', 'rb') as f:
    server = pickle.load(f)

with open('clients.pkl', 'rb') as f:
    clients = pickle.load(f)